In [2]:
import pandas as pd
data=pd.read_csv('/content/RickAndMortyScripts.csv')
data.head()


,index,season no.,episode no.,episode name,name,line
0,0,1,1,Pilot,Rick,Morty! You gotta come on. Jus'... you gotta co...
1,1,1,1,Pilot,Morty,"What, Rick? What’s going on?"
2,2,1,1,Pilot,Rick,"I got a surprise for you, Morty."
3,3,1,1,Pilot,Morty,It's the middle of the night. What are you tal...
4,4,1,1,Pilot,Rick,"Come on, I got a surprise for you. Come on, h..."


In [3]:
conversations = []
for i in range(len(data) - 1):
    if data['name'][i] != data['name'][i + 1]:  # Ensuring it's a back-and-forth conversation
        prompt = f"{data['name'][i]}: {data['line'][i]}"
        response = f"{data['name'][i + 1]}: {data['line'][i + 1]}"
        conversations.append({"prompt": prompt, "response": response})

In [30]:

from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
tokenizer = GPT2Tokenizer.from_pretrained("af1tang/personaGPT")
model = GPT2LMHeadModel.from_pretrained("af1tang/personaGPT")
if torch.cuda.is_available():
    model = model.cuda()
## utility functions ##
flatten = lambda l: [item for sublist in l for item in sublist]

def to_data(x):
    if torch.cuda.is_available():
        x = x.cpu()
    return x.data.numpy()

def to_var(x):
    if not torch.is_tensor(x):
        x = torch.Tensor(x)
    if torch.cuda.is_available():
        x = x.cuda()
    return x

def display_dialog_history(dialog_hx):
    for j, line in enumerate(dialog_hx):
        msg = tokenizer.decode(line)
        if j %2 == 0:
            print(">> User: "+ msg)
        else:
            print("Bot: "+msg)
            print()

def generate_next(bot_input_ids, do_sample=True, top_k=10, top_p=0.92,
                  max_new_tokens=50, pad_token=tokenizer.eos_token_id):
    # Limit input length to the model's max capacity
    max_model_length = model.config.max_position_embeddings
    if bot_input_ids.shape[-1] > max_model_length:
        bot_input_ids = bot_input_ids[:, -max_model_length:]

    # Generate response with limited new tokens
    full_msg = model.generate(bot_input_ids, do_sample=do_sample,
                              top_k=top_k, top_p=top_p,
                              max_new_tokens=max_new_tokens, pad_token_id=pad_token)

    msg = to_data(full_msg.detach()[0])[bot_input_ids.shape[-1]:]
    return msg


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [33]:
personas_rick = []
for i in range(len(data)-1):
  if data['name'][i]=="Rick":
    line = data['line'][i] + tokenizer.eos_token
    personas_rick.append(line)

#  Encode the personas as a single prompt
personas_rick = ''.join(['<|p2|>'] + personas_rick + ['<|sep|>'] + ['<|start|>'])
token_limit = 800   # Token limit


# Tokenize the formatted input
input_ids = tokenizer.encode(personas_rick)

input_ids=input_ids[:token_limit]


In [34]:
# converse for 6 turns
dialog_hx = []
for step in range(6):
    # encode the user input
    user_inp = tokenizer.encode(input(">> User: ") + tokenizer.eos_token)
    # append to the chat history
    dialog_hx.append(user_inp)

    # generated a response while limiting the total chat history to 1000 tokens,
    bot_input_ids = to_var([input_ids + flatten(dialog_hx)]).long()
    msg = generate_next(bot_input_ids)
    dialog_hx.append(msg)
    print("Bot: {}".format(tokenizer.decode(msg, skip_special_tokens=True)))


>> User: hi what is your name?
Bot: no, Morty.
>> User: no as in you don't remember your name or no as in you don't have one?
Bot: you, Morty.
>> User: ok, that was weird. where are we going?
Bot: good luck
>> User: so am i going alone, don't leave me rick
Bot: that's that, Morty. you
>> User: ok, what is the destination?
Bot: you are we gonna be careful! you, mmmmmm, i think i will be a man.
>> User: is it going to be dangerous?
Bot: do you gonna get that's just be a little girl, like, i'm the little sister of the world, and that you?
